In [ ]:
import random
import numpy as np
class Jeu():

  def __init__(self, training_on, epoch, eps, display: True, predict_mode: False, values):
    self.players()
    if training_on:
      for i in range(epoch):
        self.historique()
        self.depart()
        self.action()
        if epoch % 10 == 0:
          self.eps = self.eps * 0.95
    else:
      self.historique()
      self.depart()
      self.action()
    if predict_mode:
        training_on = False
        self.trainable = False
        self.state_values= values
        self.eps = 0
        self.p2_is_human = True
        self.historique()
        self.depart()

  def players(self):
    self.p1_is_human = False
    self.p2_is_human = False
    self.trainable = True
    if self.trainable:
      self.state_values = {f"V{i}": 0 for i in range(1, 13)}
      self.eps = eps

  def historique(self):
    self.p1_hist = { "Tour": [],
               "S": [],
                "S'":[],
                "r": []
                }
    self.p2_hist = { "Tour": [],
               "S": [],
                "S'":[],
                "r": []
                }

  def depart(self):
    self.tour = 0
    self.loss = -1
    self.win = 1
    self.no = 0
    self.allumettes = 12
    self.p1 = True
    if display: print(f"|" * self.allumettes)
    if predict_mode: self.action()

  def reset(self):
    self.depart()


  def update_values(self):
    lr = 0.1
    temp = 0
    for element in range(1,len(self.p1_hist["Tour"]) + 1):
      state_p1 = self.p1_hist["S"][-1 * element]
      if element == 1:
        self.state_values[f"V{state_p1}"] = self.state_values[f"V{state_p1}"] + lr*( self.p1_hist["r"][-1]- self.state_values[f"V{state_p1}"])
        temp_p1 = self.state_values[f"V{state_p1}"]
      else:
        self.state_values[f"V{state_p1}"] = self.state_values[f"V{state_p1}"] + lr*( temp_p1 - self.state_values[f"V{state_p1}"])
        temp_p1 = self.state_values[f"V{state_p1}"]
    for element in range(1,len(self.p2_hist["Tour"]) + 1):
      state_p2 = self.p2_hist["S"][-1 * element]
      if element == 1:
        self.state_values[f"V{state_p2}"] = self.state_values[f"V{state_p2}"] + lr*( self.p2_hist["r"][-1]- self.state_values[f"V{state_p2}"])
        temp_p2 = self.state_values[f"V{state_p2}"]
      else:
        self.state_values[f"V{state_p2}"] = self.state_values[f"V{state_p2}"] + lr*( temp_p2 - self.state_values[f"V{state_p2}"])
        temp_p2 = self.state_values[f"V{state_p2}"]


  def is_won(self):
    if self.allumettes <= 0:
      self.allumettes = 0
      if display: print('P2 won') if self.p1 else print('P1 won')
      if self.p1:
        self.p1_hist["r"].append(-1)
        self.p2_hist["r"].append(1)
      else:
        self.p1_hist["r"].append(1)
        self.p2_hist["r"].append(-1)

      self.p1_hist["S'"].append(0)
      self.p2_hist["S'"].append(0)
      if self.trainable:
        self.update_values()
      self.reset()
    else:
      if self.p1 and self.tour > 1:
        self.p2_hist["r"].append(0)
        self.p2_hist["S'"].append(self.allumettes)
      elif not self.p1:
        self.p1_hist["r"].append(0)
        self.p1_hist["S'"].append(self.allumettes)
      if display: print(f"|" * self.allumettes)
      self.p1 = not self.p1
      self.action()

  def bot_choice(self,predict_mode):
      if not predict_mode:
        self.eps = max(self.eps, 0.05)
      if self.p1:
        state = self.p1_hist["S"][-1]
        possible_action = [self.state_values[f"V{max(state - 1,1)}"], self.state_values[f"V{max(state - 2,1)}"], self.state_values[f"V{max(state - 3,1)}"]]
      else:
        state = self.p2_hist["S"][-1]
        possible_action = [self.state_values[f"V{max(state - 1,1)}"], self.state_values[f"V{max(state - 2,1)}"], self.state_values[f"V{max(state - 3,1)}"]]
      if random.random() < self.eps:
          bot_action = random.randint(1, 3)
      else:
          bot_action = possible_action.index(min(possible_action)) + 1
      self.step = bot_action


  def action(self):
    self.tour += 1
    if self.p1:
      self.p1_hist["Tour"].append(self.tour)
      self.p1_hist["S"].append(self.allumettes)
      if display: print(" P1 plays:")
      if self.p1_is_human:
        action = int(input())
        self.step = action
      else:
        self.bot_choice(predict_mode)
      self.allumettes -= self.step
      self.is_won()

    else:
      self.p2_hist["Tour"].append(self.tour)
      self.p2_hist["S"].append(self.allumettes)
      if display: print(" P2 plays:")
      if self.p2_is_human:
        action = int(input())
        self.step = action
      else:
        self.bot_choice(predict_mode)
      self.allumettes -= self.step
      self.is_won()





In [ ]:
training_on = True
epoch = 10000
eps = 0.996
display = False
predict_mode = False
values = []
test = Jeu(training_on,epoch,eps,display, predict_mode, values)

In [ ]:
training_on = True
epoch = 0
eps = 0
display = True
predict_mode = True
values = test.state_values
test1 = Jeu(training_on,epoch,eps,display, predict_mode, values)

||||||||||||
 P1 plays:
|||||||||
 P2 plays:


KeyboardInterrupt: Interrupted by user